In [0]:
#Instalando as bibliotecas necessárias
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install -q pyspark

    100% |████████████████████████████████| 215.7MB 110kB/s 
    100% |████████████████████████████████| 204kB 28.2MB/s 


In [0]:
#Variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.1-bin-hadoop2.7"

In [0]:
#Encontrando e instanciando o Spark Session
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
from pyspark.sql import *

In [0]:
dataset = spark.read.format("json").option("multiLine",True).load("sample_data/anscombe.json")
dataset.columns

['Series', 'X', 'Y']

In [0]:
dataset.show(10)

+------+----+-----+
|Series|   X|    Y|
+------+----+-----+
|     I|10.0| 8.04|
|     I| 8.0| 6.95|
|     I|13.0| 7.58|
|     I| 9.0| 8.81|
|     I|11.0| 8.33|
|     I|14.0| 9.96|
|     I| 6.0| 7.24|
|     I| 4.0| 4.26|
|     I|12.0|10.84|
|     I| 7.0| 4.81|
+------+----+-----+
only showing top 10 rows



In [0]:
dataset.printSchema()

root
 |-- Series: string (nullable = true)
 |-- X: double (nullable = true)
 |-- Y: double (nullable = true)



In [0]:
from pyspark.sql import functions as F
dataset_agrupado = dataset.groupBy("Series") \
                          .agg(F.avg("X").alias("X_agrupado"), F.avg("Y").alias("Y_agrupado"))  \
                          .orderBy("Series")

dataset_agrupado.show()

+------+----------+-----------------+
|Series|X_agrupado|       Y_agrupado|
+------+----------+-----------------+
|     I|       9.0|              7.5|
|    II|       9.0|7.500909090909091|
|   III|       9.0|7.500000000000001|
|    IV|       9.0| 7.50090909090909|
+------+----------+-----------------+



In [0]:
dataset_agrupado.explain()

== Physical Plan ==
*(3) Sort [Series#0 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(Series#0 ASC NULLS FIRST, 200)
   +- *(2) HashAggregate(keys=[Series#0], functions=[avg(X#1), avg(Y#2)])
      +- Exchange hashpartitioning(Series#0, 200)
         +- *(1) HashAggregate(keys=[Series#0], functions=[partial_avg(X#1), partial_avg(Y#2)])
            +- *(1) FileScan json [Series#0,X#1,Y#2] Batched: false, Format: JSON, Location: InMemoryFileIndex[file:/content/sample_data/anscombe.json], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Series:string,X:double,Y:double>
